In [378]:
import numpy as np
import os
from abc import abstractmethod
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [379]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [380]:
def assert_preds_correct(your_preds, sklearn_preds) -> bool:
    return np.abs(your_preds - sklearn_preds).sum() == 0

In [381]:
def assert_probs_correct(your_probs, sklearn_probs) -> bool:
    return np.abs(your_probs - sklearn_probs).mean() < 1e-3

In [382]:
# Не изменяйте код этого класса!
class NaiveBayes:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.params = dict()
        
    # --- PREDICTION ---
        
    def predict(self, x, return_probs=False):
        """
        x - np.array размерности [N, dim], 
        где N - количество экземпляров данных, 
        dim -размерность одного экземпляра (количество признаков).
        
        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs)
            )
        
        if return_probs:
            return np.array(preds, dtype='float32')
        
        return np.array(preds, dtype='int32')
    
    # Совет: вниманительно изучите файл подсказок к данной лабораторной
    # и сопоставьте код с описанной математикой байесовского классификатора.
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.
        
        x - np.array размерности dim.
        
        Возвращает номер класса, которому принадлежит x.
        """
        assert len(x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        marginal_prob = self.compute_marginal_probability(x)  # P(x) - безусловная вероятность появления x
        
        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            prior = self.compute_prior(c)               # P(c) - априорная вероятность (вероятность появления класса)
            likelihood = self.compute_likelihood(x, c)  # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            
            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = prior * likelihood / marginal_prob
            probs.append(prob)
            
        if return_probs:
            return probs
        
        return np.argmax(probs)
    
    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass
    
    # Вычисляет P(c) - априорная вероятность появления класса c.
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass
    
    # Вычисляет P(x|c) - вероятность наблюдения экземпляра x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass
    
    # --- FITTING ---
    
    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)
    
    @abstractmethod
    def _estimate_prior(self, y):
        pass
    
    @abstractmethod
    def _estimate_params(self, x, y):
        pass

## 1. Наивный классификатор Байеса: гауссово распределение

Напишите недостающий код, создайте и обучите модель. 

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

In [383]:
x_train, y_train, x_test, y_test = load_data('gauss')

In [384]:
# P(C_k|x) = P(x|theta) * P(C_k) / P(x)

class NaiveGauss(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        marginal_prob = 0.0
        for c in range(self.n_classes):
            prior = self.compute_prior(c)
            likelihood = self.compute_likelihood(x, c)
            marginal_prob += prior * likelihood
        return marginal_prob
    
    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        return self.params['prior'][c]
    
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        mean = self.params['mean'][c]
        std = self.params['std'][c]
        exponent = np.exp(-((x - mean) ** 2) / (2 * std ** 2))
        return np.prod((1 / (np.sqrt(2 * np.pi * std ** 2))) * exponent)
    
    # --- FITTING ---
    
    def _estimate_prior(self, y):
        class_counts = np.bincount(y)
        total_samples = len(y)
        self.params['prior'] = class_counts / total_samples
        
    
    def _estimate_params(self, x, y):
        self.params['mean'] = []
        self.params['std'] = []
        for c in range(self.n_classes):
            x_c = x[y == c]
            class_mean = np.mean(x_c, axis=0)
            self.params['mean'].append(class_mean)
            class_std = np.std(x_c, axis=0)
            self.params['std'].append(class_std)

In [385]:
# Создайте и обучите модель
model = NaiveGauss(n_classes=len(np.unique(y_train)))
model.fit(x_train, y_train)

preds = model.predict(x_test)
probs = model.predict(x_test, return_probs=True)

In [386]:
# Оцените качество модели
my_accuracy = accuracy_score(y_test, preds)
my_f1_score = f1_score(y_test, preds, average='weighted')
my_recall_score = recall_score(y_test, preds, average='weighted')
my_precision_score = precision_score(y_test, preds, average='weighted')

print(f"Accuracy моего наивного Байеса: {my_accuracy}")
print(f"F1 моего наивного Байеса: {my_f1_score}")
print(f"Recall моего наивного Байеса: {my_recall_score}")
print(f"Precision моего наивного Байеса: {my_precision_score}")

Accuracy моего наивного Байеса: 1.0
F1 моего наивного Байеса: 1.0
Recall моего наивного Байеса: 1.0
Precision моего наивного Байеса: 1.0


In [387]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
from sklearn.naive_bayes import GaussianNB

sklearn_model = GaussianNB()
sklearn_model.fit(x_train, y_train)
sklearn_preds = sklearn_model.predict(x_test)
sklearn_probs = sklearn_model.predict_proba(x_test)

sklearn_accuracy = accuracy_score(y_test, sklearn_preds)
sklearn_f1_score = f1_score(y_test, sklearn_preds, average='weighted')
sklearn_recall_score = recall_score(y_test, sklearn_preds, average='weighted')
sklearn_precision_score = precision_score(y_test, sklearn_preds, average='weighted')

print(f"Accuracy модели из sklearn наивного Байеса: {sklearn_accuracy}")
print(f"F1 модели из sklearn наивного Байеса: {sklearn_f1_score}")
print(f"Recall модели из sklearn наивного Байеса: {sklearn_recall_score}")
print(f"Precision модели из sklearn наивного Байеса: {sklearn_precision_score}")

Accuracy модели из sklearn наивного Байеса: 1.0
F1 модели из sklearn наивного Байеса: 1.0
Recall модели из sklearn наивного Байеса: 1.0
Precision модели из sklearn наивного Байеса: 1.0


In [388]:
#тут добавила оценку из пунктов оценки :)

if assert_preds_correct(preds, sklearn_preds):
    print("Предсказанные классы совпадают с моделью sklearn")
else:
    print("Предсказанные классы НЕ совпадают с моделью sklearn")

if assert_probs_correct(probs, sklearn_probs):
    print("Предсказанные вероятности совпадают с моделью sklearn")
else:
    print("Предсказанные вероятности НЕ совпадают с моделью sklearn")

Предсказанные классы совпадают с моделью sklearn
Предсказанные вероятности совпадают с моделью sklearn


In [389]:
##  Итоги:
## 1. Совпали метрики, значит и моя модель, и модель из sklearn
## 2. Совпали предсказанные классы у моей модели и у модели sklearn
## 3. Совпали значения предсказанных вероятностей принадлежности классами у моей модели и у модели sklearn

## 2. Доп. задания (любое на выбор, опционально)

### 2.1  Упрощение наивного классификатора Байеса для гауссова распределения

Уберите из класса NaiveBayes 'лишние' вычисления и удалите код, что соответствует этим вычислениям. Под 'лишним' подразумеваются вещи, что не влияют на итоговое решение о принадлежности классу (значения вероятностей при этом могу стать некорректными, но в данном задании это допустимо).

Напишите в клетке ниже код упрощенного 'классификатора Гаусса' и убедитесь, что его ответы (не значения вероятностей) совпадают с ответами классификатора из задания 1. Для сравнения ответов используйте функцию `assert_preds_correct`.

Указание: работайте в предположении, что классы равновероятны.

Подсказка: упростить необходимо метод `predict_single`.

In [390]:
class SimplifiedNaiveGauss(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        return 1.0
    
    def compute_prior(self, c) -> float:
        return 1.0 / self.n_classes
    
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        mean = self.params['mean'][c]
        std = self.params['std'][c]
        exponent = np.exp(-((x - mean) ** 2) / (2 * std ** 2))
        return np.prod((1 / (np.sqrt(2 * np.pi * std ** 2))) * exponent)
    
    # --- FITTING ---
    
    def _estimate_prior(self, y):
        pass
    
    def _estimate_params(self, x, y):
        self.params['mean'] = []
        self.params['std'] = []
        for c in range(self.n_classes):
            x_c = x[y == c]
            class_mean = np.mean(x_c, axis=0)
            self.params['mean'].append(class_mean)
            class_std = np.std(x_c, axis=0)
            self.params['std'].append(class_std)
    
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.
        
        x - np.array размерности dim.
        
        Возвращает номер класса, которому принадлежит x.
        """
        assert len(x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        
        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            prior = self.compute_prior(c)               # P(c) - априорная вероятность (вероятность появления класса)
            likelihood = self.compute_likelihood(x, c)  # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            
            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c)
            prob = prior * likelihood
            probs.append(prob)
            
        if return_probs:
            return probs
        
        return np.argmax(probs)

In [391]:
# Создайте и обучите модель
simplified_model = SimplifiedNaiveGauss(n_classes=len(np.unique(y_train)))
simplified_model.fit(x_train, y_train)

simplified_preds = simplified_model.predict(x_test)
simplified_probs = simplified_model.predict(x_test, return_probs=True)

In [392]:
# Оцените качество модели
simplified_accuracy = accuracy_score(y_test, simplified_preds)
simplified_f1_score = f1_score(y_test, simplified_preds, average='weighted')
simplified_recall_score = recall_score(y_test, simplified_preds, average='weighted')
simplified_precision_score = precision_score(y_test, simplified_preds, average='weighted')

print(f"Accuracy упрощенной модели наивного Байеса: {simplified_accuracy}")
print(f"F1 упрощенной модели наивного Байеса: {simplified_f1_score}")
print(f"Recall упрощенной модели наивного Байеса: {simplified_recall_score}")
print(f"Precision упрощенной модели наивного Байеса: {simplified_precision_score}")

Accuracy упрощенной модели наивного Байеса: 1.0
F1 упрощенной модели наивного Байеса: 1.0
Recall упрощенной модели наивного Байеса: 1.0
Precision упрощенной модели наивного Байеса: 1.0


In [393]:
if assert_preds_correct(preds, simplified_preds):
    print("Предсказанные классы совпадают с упрощенной модели")
else:
    print("Предсказанные классы НЕ совпадают с упрощенной модели")

if assert_probs_correct(probs, simplified_probs):
    print("Предсказанные вероятности совпадают с упрощенной модели")
else:
    print("Предсказанные вероятности НЕ совпадают с упрощенной модели")

Предсказанные классы совпадают с упрощенной модели
Предсказанные вероятности НЕ совпадают с упрощенной модели


In [394]:
# Объясните в комментариях к этой клетке суть проделанных изменений: почему удаленный код является лишним?

#Метод compute_marginal_probability теперь возвращает константу 1.0, так как безусловная вероятность P(x) не влияет на выбор класса
#Если все классы равновероятны, то априорная вероятность для каждого класса равна 1.0 / self.n_classes
#Метод _estimate_prior больше не нужен, т.к. мы предполагаем равновероятность классов
#В методе predict_single убрала деление на безусловную вероятность P(x), т.к. она не влияет на выбор класса с максимальной вероятностью

### 2.1  Наивный классификатор Байеса: мультиномиальное распределения

Напишите недостающий код, создайте и обучите модель.

Подсказка: в определении функции правдоподобия много факториалов. Для избежания численного переполнения посчитайте сначала логарифм функции правдоподобия (на бумаге), после примените экспоненту для получения значения вероятности.

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

Сложность: математический гений.

In [395]:
x_train, y_train, x_test, y_test = load_data('multinomial')

In [396]:
class NaiveMultinomial(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        """Вычисляет безусловную вероятность P(x) как сумму по всем классам."""
        marginal_prob = 0.0
        for c in range(self.n_classes):
            prior = self.compute_prior(c)
            likelihood = self.compute_likelihood(x, c)
            marginal_prob += prior * likelihood
        return marginal_prob

    def compute_prior(self, c) -> float:
        """Возвращает априорную вероятность класса c."""
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Сумма априорных вероятностей должна быть равна 1, но равна {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Индекс класса должен быть < {self.n_classes}, но получен {c}.'
        return self.params['prior'][c]

    def compute_likelihood(self, x, c) -> float:
        """Вычисляет правдоподобие P(x|c) для заданного класса c и вектора признаков x."""
        assert c < self.n_classes, f'Индекс класса должен быть < {self.n_classes}, но получен {c}.'
        total_sum = sum(x)
        log_likelihood = np.sum(np.log(np.arange(total_sum, 0, -1))) - sum(np.log(xi) for xi in x if xi > 0)
        for feature_idx, feature_value in enumerate(x):
            if feature_value > 0:
                log_likelihood += feature_value * log(self.params['likelihood'][c][feature_idx])
        return exp(log_likelihood)

    def _estimate_prior(self, y):
        """Оценивает априорные вероятности P(c) на основе обучающей выборки."""
        self.params['prior'] = np.bincount(y) / len(y)

    def _estimate_params(self, x, y):
        """Оценивает параметры правдоподобия P(x|c) с использованием сглаживания Лапласа."""
        self.params['likelihood'] = []
        for c in range(self.n_classes):
            x_c = x[y == c]
            feature_sums = np.sum(x_c, axis=0)
            total_sum = np.sum(feature_sums)
            likelihood = (feature_sums + 1) / (total_sum + x.shape[1])
            self.params['likelihood'].append(likelihood)

In [397]:
# Создайте и обучите модель
nm_model = NaiveMultinomial(n_classes=len(np.unique(y_test)))
nm_model.fit(x_train, y_train)

nm_preds= nm_model.predict(x_test)
nm_probs= nm_model.predict(x_test, return_probs=True)

In [398]:
# Оцените качество модели
nm_accuracy = accuracy_score(y_test, nm_preds)
nm_f1_score = f1_score(y_test, nm_preds, average='weighted')
nm_recall_score = recall_score(y_test, nm_preds, average='weighted')
nm_precision_score = precision_score(y_test, nm_preds, average='weighted')

print(f"Accuracy модели с мультиномиальным распределение наивного Байеса: {nm_accuracy}")
print(f"F1 модели с мультиnm_predsномиальным распределени наивного Байеса: {nm_f1_score}")
print(f"Recall модели с мультиномиальным распределени наивного Байеса: {nm_recall_score}")
print(f"Precision модели с мультиномиальным распределени наивного Байеса: {nm_precision_score}")

Accuracy модели с мультиномиальным распределение наивного Байеса: 0.8173076923076923
F1 модели с мультиnm_predsномиальным распределени наивного Байеса: 0.8172889197856059
Recall модели с мультиномиальным распределени наивного Байеса: 0.8173076923076923
Precision модели с мультиномиальным распределени наивного Байеса: 0.8173373404436152


In [399]:
# Сравните вашу модель с аналогом sklearn (MultinomialNB)
from sklearn.naive_bayes import MultinomialNB

sklearn_model = MultinomialNB()
sklearn_model.fit(x_train, y_train)

sklearn_preds = sklearn_model.predict(x_test)
sklearn_probs = sklearn_model.predict_proba(x_test)

sklearn_accuracy = accuracy_score(y_test, sklearn_preds)
sklearn_f1_score = f1_score(y_test, sklearn_preds, average='weighted')
sklearn_recall_score = recall_score(y_test, sklearn_preds, average='weighted')
sklearn_precision_score = precision_score(y_test, sklearn_preds, average='weighted')

print(f"Accuracy модели из sklearn наивного Байеса: {sklearn_accuracy}")
print(f"F1 модели из sklearn наивного Байеса: {sklearn_f1_score}")
print(f"Recall модели из sklearn наивного Байеса: {sklearn_recall_score}")
print(f"Precision модели из sklearn наивного Байеса: {sklearn_precision_score}")


Accuracy модели из sklearn наивного Байеса: 0.8173076923076923
F1 модели из sklearn наивного Байеса: 0.8172889197856059
Recall модели из sklearn наивного Байеса: 0.8173076923076923
Precision модели из sklearn наивного Байеса: 0.8173373404436152


In [400]:
if assert_preds_correct(nm_preds, sklearn_preds):
    print("Предсказанные классы совпадают с моделью из sklearn")
else:
    print("Предсказанные классы НЕ совпадают с моделью из sklearn")

if assert_probs_correct(nm_probs, sklearn_probs):
    print("Предсказанные вероятности совпадают с моделью из sklearn")
else:
    print("Предсказанные вероятности НЕ совпадают с моделью из sklearn")

Предсказанные классы совпадают с моделью из sklearn
Предсказанные вероятности совпадают с моделью из sklearn
